In [ ]:
# default_exp shapley_values

In [ ]:
#export
import numpy as np
import pandas as pd
from itertools import combinations
from math import factorial
from tqdm import tqdm

# Shapley Value

> Calculate the exact Shapley Values for an individual $x$ in a game based on a reference $r$ and the reward function $fc$.

## Theory

### Shapley Value definition

In Collaborative Game Theory, Shapley Values ([Shapley,1953]) can distribute a reward among players in a fairly way according to their contribution to the win in a cooperative game. We note $\mathcal{M}$ a set of $d$ players. Moreover, $v : P(\mathcal{M}) \rightarrow R_v$ a reward function such that $v(\emptyset) = 0$. The range $R_v$ can be $\Re$ or a subset of $\Re$. $P(\mathcal{M})$ is a family of sets over $\mathcal{M}$. If $S \subset \mathcal{M}\text{, } v(S)$ is the amount of wealth produced by coalition $S$ when they cooperate.

The Shapley Value of a player $j$ is a fair share of the global wealth $v(\mathcal{M})$ produced by all players together:

$$\phi_j(\mathcal{M},v) = \sum_{S \subset \mathcal{M}\backslash \{j\}}\frac{(d -|S| - 1)!|S|!}{d!}\left(v(S\cup \{j\}) - v(S)\right),$$

with $|S| = \text{cardinal}(S)$, i.e. the number of players in coalition $S$.

### Shapley Values as contrastive local attribute importance in Machine Learning

Let be $X^*\subset\Re^d$ a dataset of individuals where a Machine Learning model $f$ is trained and/or tested and $d$  the dimension of $X^*$. $d>1$ else we do not need to compute Shapley Value. We consider the attribute importance of an individual $\mathbf{x^*} = \{x_1^*, \dots, x_d^*\} \in X^*$ according to a given reference $\mathbf{r} = \{r_1, \dots, r_d\}\in X^*$.  We're looking for $\boldsymbol{\phi}=(\phi_j)_{j\in\{1, \dots, d\}}\in \Re^d$ such that:
$$ \sum_{j=1}^{d} \phi_j = f(\mathbf{x^*}) - f(\mathbf{r}), $$ 
where $\phi_j$ is the attribute contribution of feature indexed $j$.  We loosely identify each feature by its column number. Here the set of players $\mathcal{M}=\{1, \dots, d\}$ is the feature set.

In Machine Learning, a common choice for the reward is $ v(S) = \mathbb{E}[f(X) | X_S = \mathbf{x_S^*}]$, where $\mathbf{x_S^*}=(x_j^*)_{j\in S}$ and $X_S$ the element of $X$ for the coalition $S$. 
For any $S\subset\mathcal{M}$, let's define $ z(\mathbf{x^*},\mathbf{r},S)$ such that $z(\mathbf{x^*},\mathbf{r},\emptyset) = \mathbf{r}$, \ $z(\mathbf{x^*},\mathbf{r},\mathcal{M}) = \mathbf{x^*}$ and

$$ z(\mathbf{x^*},\mathbf{r},S) = (z_1,\dots, z_d) \text{ with } z_i =  x_i^* \text{ if } i \in S \text{ and } r_i  \text{ otherwise }$$ 

As explain in [Merrick,2019], each reference $\textbf{r}$ sets a single-game with $ v(S) = f(z(\mathbf{x^*},\mathbf{r},S)) - f(\mathbf{r}) $, $v(\emptyset) = 0 $ and $v(\mathcal{M}) = f(\mathbf{x^*}) - f(\mathbf{r}) $.

### References

[Shapley,1953] _A value for n-person games_. Lloyd S Shapley. In Contributions to the Theory of Games, 2.28 (1953), pp. 307 - 317.

[Merrick,2019] _The Explanation Game: Explaining Machine Learning Models with Cooperative Game Theory_. Luke Merrick, Ankur Taly, 2019.

## Function 

__Parameters__

* `x`: pandas Series. The instance $\mathbf{x^*}$ for which we want to calculate Shapley value of each attribute,

* `fc`: python function. The reward function $v$,

* `r`: pandas Series. The reference $\mathbf{r}$. The Shapley values (attribute importance) is a contrastive explanation according to this individual.

__Returns__

* `Φ`: pandas Series. Shapley values of each attribute

In [ ]:
#export
def ShapleyValues(x, fc, r):
    """
    Calculate the exact Shapley Values for an individual x
    in a game based on a reference r and the reward function fc.
    """

    # Get general information
    feature_names = list(x.index)
    d = len(feature_names) # dimension
    set_features = set(feature_names)

    # Store Shapley Values in a pandas Series
    Φ = pd.Series(np.zeros(d), index=feature_names)

    # Start computation (number of coalitions: 2**d - 1)
    for cardinal_S in tqdm(range(0, d)):
        # weight
        ω = factorial(cardinal_S) * (factorial(d - cardinal_S - 1))
        ω /= factorial(d)
        # iter over all combinations of size cardinal_S
        for S in combinations(feature_names, cardinal_S):
            S = list(S)
            z_S = np.array([x[col] if col in S else r[col] for col in feature_names])
            f_S = fc(z_S)
            # Consider only features outside of S
            features_out_S = set_features - set(S)
            for j in features_out_S:
                S_union_j = S + [j]
                z_S_union_j = np.array([x[col] if col in S_union_j else r[col] for col in feature_names])
                f_S_union_j = fc(z_S_union_j)
                # Update Shapley value of attribute i
                Φ[j] += ω * (f_S_union_j - f_S)

    return Φ

## Example

We use a simulated dataset from the book _Elements of Statistical Learning_ ([hastie,2009], the Radial example). $X_1, \dots , X_{d}$ are standard independent Gaussian. The model is determined by:

$$ Y = \prod_{j=1}^{d} \rho(X_j), $$

where $\rho\text{: } t \rightarrow \sqrt{(0.5 \pi)} \exp(- t^2 /2)$. The regression function $f_{regr}$ is deterministic and simply defined by $f_r\text{: } \textbf{x} \rightarrow \prod_{j=1}^{d} \phi(x_j)$. For a reference $\mathbf{r^*}$ and a target $\mathbf{x^*}$, we define the reward function $v_r^{\mathbf{r^*}, \mathbf{x^*}}$ such as for each coalition $S$, $v_r^{\mathbf{r^*}, \mathbf{x^*}}(S) = f_{regr}(\mathbf{z}(\mathbf{x^*}, \mathbf{r^*}, S)) - f_{regr}(\mathbf{r^*}).$

 [hastie,2009] _The Elements of Statistical Learning: Data Mining, Inference, and Prediction, Second Edition_. Hastie, Trevor and Tibshirani, Robert and Friedman, Jerome. Springer Series in Statistics, 2009.
	

In [ ]:
d, n_samples = 5, 100
mu = np.zeros(d)
Sigma = np.zeros((d,d))
np.fill_diagonal(Sigma, [1] * d)
X = np.random.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)
X = pd.DataFrame(X, columns=['x'+str(i) for i in range(1, d+1)])
def fc(x):
    phi_x = np.sqrt(.5 * np.pi) * np.exp(-0.5 * x ** 2)
    return np.prod(phi_x)
y = np.zeros(len(X))
for i in range(len(X)):
    y[i] = fc(X.values[i])
n = 2**d - 2
print("dimension = {0} ; nb of coalitions = {1}".format(str(d), str(n)))

dimension = 5 ; nb of coalitions = 30


In [ ]:
idx_r, idx_x = np.random.choice(np.arange(len(X)), size=2, replace=False)
r = X.iloc[idx_r,:]
x = X.iloc[idx_x,:]

In [ ]:
true_shap = ShapleyValues(x=x, fc=fc, r=r)

100%|██████████| 5/5 [00:00<00:00, 379.23it/s]


In [ ]:
true_shap

x1    0.128851
x2    0.024963
x3    0.122699
x4   -0.057562
x5    0.008842
dtype: float64

## Tests

In [ ]:
r_pred = fc(r.values)
x_pred = fc(x.values)
v_M = x_pred - r_pred

In [ ]:
assert np.abs(true_shap.sum() - v_M) <= 1e-10 

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted index.ipynb.
Converted inspector.ipynb.
Converted monte_carlo_shapley.ipynb.
Converted sgd_shapley.ipynb.
Converted shapley_values.ipynb.
